In [1]:
import pandas as pd
import numpy as np
import re
import random
import sys

from keras import layers
from keras.models import Sequential
from keras import optimizers

/Users/jinli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Preview

In [2]:
data_dir = "Data/transaction4.csv"
data = pd.read_csv(data_dir, error_bad_lines=False)

b'Skipping line 4403: expected 8 fields, saw 11\nSkipping line 14589: expected 8 fields, saw 10\nSkipping line 14605: expected 8 fields, saw 9\nSkipping line 14644: expected 8 fields, saw 24\nSkipping line 14670: expected 8 fields, saw 11\nSkipping line 22666: expected 8 fields, saw 9\nSkipping line 31688: expected 8 fields, saw 9\nSkipping line 32444: expected 8 fields, saw 11\nSkipping line 32451: expected 8 fields, saw 12\nSkipping line 32507: expected 8 fields, saw 13\nSkipping line 32508: expected 8 fields, saw 13\nSkipping line 34330: expected 8 fields, saw 10\nSkipping line 34364: expected 8 fields, saw 9\nSkipping line 40110: expected 8 fields, saw 23\nSkipping line 42060: expected 8 fields, saw 9\nSkipping line 43091: expected 8 fields, saw 34\nSkipping line 57266: expected 8 fields, saw 14\nSkipping line 62530: expected 8 fields, saw 11\nSkipping line 64504: expected 8 fields, saw 9\n'
b'Skipping line 69553: expected 8 fields, saw 9\nSkipping line 75688: expected 8 fields, sa

b'Skipping line 1447044: expected 8 fields, saw 30\nSkipping line 1449921: expected 8 fields, saw 9\nSkipping line 1452852: expected 8 fields, saw 9\nSkipping line 1456360: expected 8 fields, saw 10\nSkipping line 1470357: expected 8 fields, saw 9\nSkipping line 1486157: expected 8 fields, saw 9\nSkipping line 1492016: expected 8 fields, saw 9\nSkipping line 1499913: expected 8 fields, saw 9\n'
b'Skipping line 1517925: expected 8 fields, saw 9\nSkipping line 1559314: expected 8 fields, saw 9\nSkipping line 1568491: expected 8 fields, saw 9\nSkipping line 1569058: expected 8 fields, saw 11\nSkipping line 1569749: expected 8 fields, saw 13\n'
b'Skipping line 1606444: expected 8 fields, saw 9\nSkipping line 1620271: expected 8 fields, saw 9\nSkipping line 1624144: expected 8 fields, saw 9\nSkipping line 1626359: expected 8 fields, saw 9\nSkipping line 1635375: expected 8 fields, saw 9\nSkipping line 1636114: expected 8 fields, saw 9\nSkipping line 1636138: expected 8 fields, saw 13\n'
b'S

In [4]:
data.head()

,payer,receiver,tran_text,year,month,day,privacy setting,Unnamed: 7
0,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,9c044a510cd7333bda6da6a2a364372974a0a8a573ac3f...,:Italy:,2019,1,21,unknown,NaN
1,f0bf2b3ec1598e6ab6ef15ea0251328ca3731e237bdb49...,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,Heroics,2019,1,19,unknown,NaN
2,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,523c406e16c2856eefb065c2ace781a81a608b6e826ca4...,To regain Kosovo,2019,1,18,unknown,NaN
3,f0bf2b3ec1598e6ab6ef15ea0251328ca3731e237bdb49...,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,Reading at a club,2019,1,18,unknown,NaN
4,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,f0bf2b3ec1598e6ab6ef15ea0251328ca3731e237bdb49...,For the Culture,2019,1,17,unknown,NaN


In [5]:
data.shape

(2155581, 8)

In [11]:
text = data['tran_text'].astype('str')[:500000]

In [14]:
all_text = ". ".join (text)

In [15]:
all_text[:5000]

':Italy:. Heroics. To regain Kosovo. Reading at a club. For the Culture. Man with the plan. Movie. Irish delight. :wine_glass:. Biryani galore. Testing reasons. yehdodnw. GiVe Me FoDd. Carbohydrates. Hnxodnnxj. Boiled meats. Movie. walk a flock a ! thank u !. Snail Thai. E. No u. Tenga. Venom. :South_Korea:. Why isnt venmo used as social media?!?!. Anra Bday. Food!. Bday dinner. Food. Somber religious meditation.. Food - uber. For posterity\'s sake.. Five guys. Bear necessities. Mhw. :cookie:. Fridge. Brita tank. Electric Bugaloo. Electric Bugaloo. Carrying my luggage"". Carrying my luggage"". Box. 2 smalls. Chon. Big boy buerg. Bbq. BBQ. Y33t. Bobert. Uber. Bobs. :movie_camera:. Avengers. Chong. Boba. :sushi:. :steaming_bowl:. Por que no?. Ching. :steaming_bowl:. Food!. Dinner yum. Family dinner. Food. Dinner. FAMILY DINNER. :face_with_steam_from_nose:. Pain. No u. Thai 55. :steaming_bowl:. Bobbing lol. Strings. Noods. :chicken::see-no-evil_monkey::hamburger::chicken::chicken::chicken

# Training

In [16]:
def vectorizing_seq (text, maxlen, step):    
    """
    :param maxlen: the length of a sequence to extract as train
    :type  maxlen: int
    :param step: sample a new sequence every n steps
    :type  step: int
    :returns: (Numpy boolean array of shape 
                    (Number of sequences, maxlen, number of distinct character),
               Numpy boolean array of shape 
                    (Number of sequences, number of distinct character),
               dictionary mapping a character to its integer placeholder)
    :rtype:   (numpy.ndarray, 
               numpy.ndarray, 
               dict)     
    """
    
    sentences = [] # hold extracted sequences
    next_chars = [] # hold next characters for each corresponding sentence

    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])

    print('Number of sequences:', len(sentences))

    chars = sorted(list(set(text)))
    print('Unique characters:', len(chars))
    char_indices = dict((char, chars.index(char)) for char in chars)
    print('Vectorization...')

    # one hot encoding the characters into binary arrays
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
        
    return x, y, char_indices

In [17]:
def create_model(x, y, maxlen, epochs, chars):
    """
    Creates and trains a model.
    :param x: Numpy boolean array of shape 
                    (Number of sequences, maxlen, number of distinct character)
    :type  x: numpy.ndarray
    :param y: Numpy boolean array of shape 
                    (Number of sequences, number of distinct character)
    :type  y: numpy.ndarray
    :param maxlen: the length of a sequence to extract as train
    :type  maxlen: int
    :param epochs: number of training iterations
    :type  epochs: int
    :param chars: list of unique characters
    :type  chars: list
    :returns: trained keras model
    :rtype:   keras.engine.sequential.Sequential
    """

    model = Sequential()
    model.add(layers.GRU(
        32,
        return_sequences=True,
        input_shape=(maxlen, len(chars)))
    )
    model.add(layers.GRU(
        64,
        input_shape=(maxlen, len(chars)))
    )
    model.add(layers.Dense(
        len(chars), 
        activation='softmax')
    )

    print(model.summary())

    optimizer = optimizers.RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    model.fit(x, y, batch_size=128, epochs=epochs)

    return (model)

In [18]:
def train_model_from_text(text, maxlen=60, step=20, epochs=10):
    """
    Given text, train the model.
    
    :param text: A string with all the text together.
    :type  text: str
    :param maxlen: the length of a sequence to extract as train
    :type  maxlen: int
    :param step: sample a new sequence every n steps
    :type  step: int
    :param epochs: number of training iterations
    :type  epochs: int
    :returns: (trained keras model,
               dictionary mapping characters to digit representations)
    :rtype:   (keras.engine.sequential.Sequential,
               dict)
    """
    
    x, y, char_indices = vectorizing_seq(text, maxlen, step)
    chars = list (char_indices.keys())
    model = create_model(x, y, maxlen, epochs, chars)
    
    return model, char_indices

In [19]:
def sample(preds, temperature=1.0):
    """
    Compute new probability distribution based on the temperature
    Higher temperature creates more randomness.
    
    :param preds: numpy array of shape (unique chars,), and elements sum to 1
    :type  preds: numpy.ndarray
    :param temperature: characterizes the entropy of probability distribution
    :type  temperature: float
    :returns: a number 0 to the length of preds - 1
    :rtype:   int
    """
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
def text_generate(model, text, char_indices, maxlen=60, temperature=1.0, textlen=400):
    """
    Generate text based on a model.
    
    :param model: trained keras model
    :type  model: keras.engine.sequential.Sequential
    :param text: lyrics
    :type  text: str
    :param char_indices: dictionary mapping a character to its integer placeholder
    :type  char_indices: dict
    :param maxlen: maximum length of the sequences
    :type  maxlen: int
    :param textlen: Number of characters of generated sequence
    :type  textlen: int
    """

    start_index = random.randint(0, len(text) - maxlen - 1) 
    generated_text = text[start_index: start_index + maxlen] 
    print('--- Generating with seed: "' + generated_text + '"')
    
    chars = list (char_indices.keys())
    
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)
    for i in range(textlen):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)

## Sample Training and Text Generation

In [21]:
# small amount for now so I can test if the code works
# train on entire dataset
sample_text = all_text[:5000]

In [22]:
maxlen = 60

model, char_indices = train_model_from_text(
    sample_text,
    maxlen=maxlen,
    step=20,
    epochs=15
)

Number of sequences: 247
Unique characters: 76
Vectorization...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 60, 32)            10464     
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                18624     
_________________________________________________________________
dense_1 (Dense)              (None, 76)                4940      
Total params: 34,028
Trainable params: 34,028
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15
247/247 [==============================] - 1s 5ms/step - loss: 4.1381
Epoch 2/15
247/247 [==============================] - 0s 850us/step - loss: 3.3469
Epoch 3/15
247/247 [==============================] - 0s 824us/step - loss: 3.2166
Epoch 4/15
247/247 [==============================] - 0s 787us/step - loss: 2.6050
Epo

In [24]:
text_generate(
    model, 
    sample_text, 
    char_indices, 
    maxlen=maxlen,
    temperature=.6,
    textlen=600
)

--- Generating with seed: "smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::"
------ temperature: 0.6
smiling_face_with_heart-eyes::smiling_face_with_heart-eyes::::iiiaaaaa:l:::oiyatoeps::::iiaaaaeae:::::iiaaaaaee:::::iiaaaaee::::oiiaaa¯eae:::::coaaaaoe::::.oa_aeer::::.oiaa.aed:::::ioaaaee::::oioaaaae.e:::::ilaaaaee:::::i aaaaaee:::: iiaaaaee:::::oopaaad.le::::ciaaaaaee:::::oyaaaae::::iioaaaeap:::::oiaaaaee:::::7iaaaane:::yiiaaaaer:::::ii_aaaea::::oiaaaae:::::gaaaae:::::c_aaaraee:::::iiaaaaaer:::::A yi ongoa_edr::::::iaaaaae::.sicaeey::::iiiaaaaepdr::::oia_aee:::::iyaaaad:.olee:::::ioaaaaaee:::::ioaaaae::::piaaaer::::.iaaaaer::.:.oiaaprerr:::::ciaaadaaer.:::cotaaee::::  yaaaaape.e:::::yaaaaare.::::cioaaaer::::.oaapaer:::::oiaaaa.ee:::::ioaaaepe::::: oa